In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(autoload_with=engine)

In [6]:
# View all of the classes that automap found
Base.classes.keys()

In [7]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [9]:
# Find the most recent date in the data set.
recent_date = session.query(func.max(Measurement.date)).first()[0]
recent_date

In [10]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results.
# Starting from the most recent data point in the database.

# Calculate the date one year from the last date in the dataset.
first_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)

# Perform a query to retrieve the data and precipitation scores.
last_12_query = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date >= first_date).all()

# Create a Pandas DataFrame from the query results and set the date column as the index.
precipitation_df = pd.DataFrame(last_12_query, columns=['date', 'precipitation'])
precipitation_df.set_index('date', inplace=True)

# Sort the dataframe by date.
precipitation_df = precipitation_df.sort_values(by='date')

# Use Pandas Plotting with Matplotlib to visualize the data.
precipitation_df.plot(rot=90, xlabel='Date', ylabel='Inches', figsize=(10, 5))


In [11]:
# Use Pandas to calculate the summary statistics for the precipitation data
summary_statistics = precipitation_df.describe()

# Exploratory Station Analysis

In [12]:
# Design a query to calculate the total number of stations in the dataset
number_stations = session.query(func.count(Station.station)).first()[0]
number_stations

In [13]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
number_stations = session.query(Measurement.station, func.count(Measurement.station)).\
                    group_by(Measurement.station).\
                    order_by(func.count(Measurement.station).desc()).all()
number_stations

In [14]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.

# Identify the most active station
most_active_station = session.query(Measurement.station)\
    .group_by(Measurement.station)\
    .order_by(func.count().desc())\
    .first()[0]

# Calculate the lowest, highest, and average temperature for the most active station
temperature_stats = session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs))\
    .filter(Measurement.station == most_active_station)\
    .all()

# Extract the results
lowest_temp, highest_temp, avg_temp = temperature_stats[0]

In [15]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram

temperature_data = session.query(Measurement.tobs)\
    .filter(Measurement.station == most_active_station)\
    .filter(Measurement.date >= first_date)\
    .all()

# Convert the query results to a list of temperatures
temperatures = [temp[0] for temp in temperature_data]

# Create a histogram using Matplotlib
plt.figure(figsize=(8, 6))
plt.hist(temperatures, bins=12, edgecolor='black')
plt.title(f"Temperature Observation Data for Station {most_active_station}\n({first_date} to 2017-08-23)")
plt.xlabel("Temperature (°F)")
plt.ylabel("Frequency")
plt.tight_layout()

# Show the histogram
plt.show()


# Close Session

In [ ]:
# Close Session
session.close()